In [ ]:
import pandas as pd
import numpy as np
import json
cars = pd.read_csv('cars.csv')
test = pd.read_csv('test.csv')
autoru = pd.read_csv('autoru.csv')

In [ ]:
color_dict = {
    'чёрный': 'black',
    'белый': 'white',
    'серый': 'gray',
    'серебристый': 'silver',
    'синий': 'blue',
    'коричневый': 'gray',
    'красный': 'red',
    'зелёный': 'green',
    'бежевый': 'brown',
    'голубой': 'blue',
    'пурпурный': 'red',
    'золотистый': 'other',
    'фиолетовый': 'violet',
    'жёлтый': 'yellow',
    'оранжевый': 'orange',
    'розовый': 'other',
    'other': 'other'
}
fuel_dict = {
    'gasoline': 'бензин',
    'diesel': 'дизель',
    'gas': 'газ',
    'hybrid-petrol': 'гибрид',
    'electric': 'электро',
    'hybrid-diesel': 'гибрид'
}
body_dict = {
    'внедорожник 5 дв.': 'suv',
    'седан': 'sedan',
    'лифтбек': 'liftback',
    'хэтчбек 5 дв.': 'hatchback',
    'универсал 5 дв.': 'universal',
    'минивэн': 'minivan',
    'купе': 'coupe',
    'компактвэн': 'minivan',
    'хэтчбек 3 дв.': 'hatchback',
    'пикап двойная кабина': 'pickup',
    'внедорожник 3 дв.': 'suv',
    'купе-хардтоп': 'coupe',
    'кабриолет': 'cabriolet',
    'фургон': 'van',
    'родстер': 'coupe',
    'микровэн': 'minivan',
    'седан-хардтоп': 'sedan',
    'пикап одинарная кабина': 'pickup',
    'пикап полуторная кабина': 'pickup',
    'внедорожник открытый': 'suv',
    'лимузин': 'limousine',
    'тарга': 'cabriolet',
    'седан 2 дв.': 'coupe',
    'фастбек': 'hatchback',
    'хэтчбек 4 дв.': 'hatchback',
    'фаэтон': 'cabriolet'
    
}
transmission_dict = {
    'автоматическая': 'automatic',
    'механическая': 'mechanical',
    'роботизированная': 'automatic',
    'вариатор': 'automatic'
}
drivetrain_dict = {
    'полный': 'all',
    'передний': 'front',
    'задний': 'rear'
}

In [ ]:
autoru.drop_duplicates(inplace=True)
autoru = autoru[autoru['Year'] != 'Year']

autoru['manufacturer_name'] = autoru['Name'].apply(lambda x:x.split()[0].upper())
autoru['manufacturer_name'] = autoru['manufacturer_name'].apply(lambda x: 'MERCEDES' if x=='MERCEDES-BENZ' else x)

autoru['Year'] = autoru['Year'].astype('int') - 1938

autoru['Odometer'] = autoru['Odometer'].apply(lambda x: ''.join(x.split())[:-2])
autoru['Odometer'] = autoru['Odometer'].apply(lambda x: 0 if x=='Нов' else x).astype('int')

def clear_autoru_price(price):
    if price[:2] == 'от':
        return int(''.join(price[2:-1].split()))
    return int(''.join(price[:-1].split()))
autoru['Price'] = autoru['Price'].apply(clear_autoru_price)
autoru = autoru[autoru['Price']<15000000]

autoru['engine_capacity'] = autoru['Engine'].apply(lambda x: x.split()[0]).astype('float')
autoru['engine_power'] = autoru['Engine'].apply(lambda x: x.split()[3]).astype('int')
autoru['engine_fuel'] = autoru['Engine'].apply(lambda x: x.split()[-1].lower())

correct_fields = autoru['Transmission'].isin(['автомат', 'механика', 'вариатор', 'робот'])
autoru.loc[~correct_fields, 'Type'] = autoru['Transmission']
autoru.loc[~correct_fields, 'Transmission'] = autoru['Color']
autoru.loc[~correct_fields, 'Color'] = 'other'

autoru['Color'] = autoru['Color'].apply(lambda x: color_dict[x])
autoru['Drive'] = autoru['Drive'].apply(lambda x:drivetrain_dict[x])
autoru['Type'] = autoru['Type'].apply(lambda x:body_dict[x])

autoru = autoru.rename(columns={'Type': 'body_type',
                                'Odometer': 'odometer_value',
                                'Transmission': 'transmission',
                                'Year': 'year_produced',
                                'Color': 'color',
                                'Drive': 'drivetrain',
                                'Price': 'price'
})

In [ ]:
def clear_eng_power(hp):
    capacity = str(hp).split()[0]
    if capacity == 'undefined':
        return 0
    return float(capacity)

def eval_features(feature_string):
    feature_count = 0
    if len(feature_string) > 5:
        for i in eval(feature_string[2:-2]):
            feature_count += len(i['values'])
        return feature_count
    else:
        return 0


cars = cars[cars['state']=='owned']
cars = cars[cars['has_warranty'] == False]
cars.dropna(inplace=True, axis=0)
cars['engine_fuel'] = cars['engine_fuel'].apply(lambda x:fuel_dict[x])
cars['manufacturer_name'] = cars['manufacturer_name'].apply(lambda x:x.upper())
cars['features_count'] = cars.iloc[:,-11:-1].sum(axis=1)

test['color'] = test['color'].apply(lambda x:color_dict[x])
test['bodyType'] = test['bodyType'].apply(lambda x:body_dict[x])
test['vehicleTransmission'] = test['vehicleTransmission'].apply(lambda x:transmission_dict[x])
test['Привод'] = test['Привод'].apply(lambda x:drivetrain_dict[x])
test['productionDate'] = test['productionDate'].apply(lambda x:int(x))
test['mileage'] = test['mileage'].apply(lambda x:int(x))
test['enginePower'] = test['enginePower'].apply(clear_eng_power)
test['engineDisplacement'] = test['engineDisplacement'].apply(clear_eng_power)
test['features_count'] = (test['Комплектация'].apply(eval_features)//8).apply(lambda x: x if x<10 else 9)

test = test.rename(columns={'bodyType': 'body_type',
                            'brand': 'manufacturer_name',
                            'mileage': 'odometer_value',
                            'vehicleTransmission': 'transmission',
                            'productionDate': 'year_produced',
                            'fuelType': 'engine_fuel',
                            'engineDisplacement': 'engine_capacity',
                            'enginePower': 'engine_power',
                            'Привод': 'drivetrain'
})

test['year_produced'] = test['year_produced'] - 1938
cars['year_produced'] = cars['year_produced'] - 1938

In [ ]:
ITERATIONS = 15000
LR         = 0.1

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
model = CatBoostRegressor(iterations = ITERATIONS,
                          #learning_rate = LR,
                          random_seed = 42,
                          depth = 6,
                          l2_leaf_reg = 1,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )

In [ ]:
features = ['body_type',#0
            'manufacturer_name',#1
            'transmission',#2
            'year_produced',#3
            'engine_fuel',#4
            'color',#5
            'drivetrain',#6
            'engine_capacity',#7
            'engine_power',#8
            'odometer_value'#9
]

X = autoru[features]
y = autoru['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [ ]:
model.fit(X, y,
         cat_features=[0,1,2,4,5,6],
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
test_pred = test[features]
pred = model.predict(test_pred)

In [ ]:
USD_RATE = 64.0817

sample_submission = pd.DataFrame()
sample_submission['price'] = pred
#sample_submission['price'] = sample_submission['price']*USD_RATE
sample_submission = sample_submission.reset_index().rename(columns={'index': 'id'})
sample_submission.to_csv(f'submission_v05.csv', index=False)
sample_submission.head()

In [ ]:
test_pred.head()

In [ ]:
random_car = ['hatchback', 'HONDA', 'automatic', 2018-1938, 'бензин', 'gray', 'front', '1.8', 2, 79, 126089]
price = model.predict(random_car)
price